# this notebook must be run with a sagemath jupyter kernel
## in a linux machine with sagemath installed run sage -n jupyter and connect to the kernel.

In [44]:
#generated using copilot, modified for clairity, and tested to match answers from https://openstax.org/books/calculus-volume-3/pages/5-6-calculating-centers-of-mass-and-moments-of-inertia
def centerOfMass(region, density, return_mass = False):
    '''   
    Calculate the center of mass for a 2D or 3D object defined by a region and a density function.
    If the object has uniform density, specify a constant value for the density function (e.g 2.5 ).
    
    also works for calculating charge densit, total heat, and other physical properties that can be defined as an integral over a region.
    
    Parameters:
    - region: A list of tuples specifying the bounds for each variable (e.g., [(x, 0, 1), (y, 0, 2)] for 2D).
    - density: The density function of the object.
    - return_mass: If True, the function will return the mass of the object along with the center of mass  
    
    Returns:
    - A tuple representing the center of mass coordinates.

    example usage: # add more cases and results from actually testing these cases
    2 dimensional case:
    >>> centerOfMass([(x, 0, 1), (y, 0, 2)], x^2+y^2 )
    3 dimensional case:
    >>> centerOfMass([(x, 0, 1), (y, 0, 2), (z, 0, 3)], x^2+y^2+z^2 )
    '''
    variables = [var[0] for var in region]

    if len(variables) == 2:  # 2D case
        x, y = variables
        mass = integral(integral(density, region[0]), region[1])
        x_cm = integral(integral(x * density, region[0]), region[1]) / mass
        y_cm = integral(integral(y * density, region[0]), region[1]) / mass
        return ((x_cm, y_cm), mass) if return_mass else (x_cm, y_cm)
    
    elif len(variables) == 3:  # 3D case
        x, y, z = variables
        mass = integral(integral(integral(density, region[0]), region[1]), region[2])
        x_cm = integral(integral(integral(x * density, region[0]), region[1]), region[2]) / mass
        y_cm = integral(integral(integral(y * density, region[0]), region[1]), region[2]) / mass
        z_cm = integral(integral(integral(z * density, region[0]), region[1]), region[2]) / mass
        return ((x_cm, y_cm, z_cm), mass) if return_mass else (x_cm, y_cm, z_cm)
    else:
        raise ValueError("The function only supports 2D or 3D objects.")



In [ ]:
# testing centerOfMass() function with rectangular reigons

print('Testing with rectangular regions:')
var('x, y, z')   
print(f'bounds:(x, 0, 6), (y, 0, 3)], density = sqrt(x*y) :\n{centerOfMass([(x, 0, 6), (y, 0, 3)], sqrt(x*y),True)}')  

print(n(24*sqrt(2)), '==',n(centerOfMass([(x, 0, 6), (y, 0, 3)], sqrt(x*y),True)[1])) # checking that answer from textbook is the same as answer from the function


print('\nbounds: (x,0,2),(y,0,3),(z,0,3)], density = z',' center of mass from homework ==(1, 3/2, 2), result ==',centerOfMass([(x,0,2),(y,0,3),(z,0,3)],z))

# testing centerOfMass() function with non-rectangular regions

print('\nTesting with non-rectangular regions:')

print('bounds: (x, 0, 1), (y, 0, sqrt(1-x^2)], density = 1 :', centerOfMass([(x, 0, 1), (y, 0, sqrt(1-x^2))], 1, True)) # validate with anwer from textbook



Testing with rectangular regions:
bounds:(x, 0, 6), (y, 0, 3)], density = sqrt(x*y) :
((18/5, 9/5), 8*sqrt(6)*sqrt(3))
33.9411254969543 == 33.9411254969543

bounds: (x,0,2),(y,0,3),(z,0,3)], density = z  center of mass from homework ==(1, 3/2, 2), result == (1, 3/2, 2)

Testing with non-rectangular regions:
bounds: (x, 0, 1), (y, 0, sqrt(1-x^2)], density = 1 : ((1/2, -1/2*(x^2 - 1)/sqrt(-x^2 + 1)), sqrt(-x^2 + 1))


In [48]:
def simulateChargeDistribution(region, resistance, current_input, boundary_conditions):
    """
    Simulate the charge distribution, total charge, and center of charge distribution
    on a 2D mesh of a material with a given resistance.

    Parameters:
    - region: A list of tuples specifying the bounds for the 2D region (e.g., [(x, 0, 1), (y, 0, 2)]).
    - resistance: A function or constant representing the resistance of the material.
    - current_input: A function specifying the current input at specific points on the mesh.
    - boundary_conditions: A dictionary specifying the boundary conditions for the simulation.

    Returns:
    - A dictionary containing:
        - 'charge_distribution': The symbolic solution for the charge distribution.
        - 'total_charge': The total charge on the plate.
        - 'center_of_charge': The center of charge distribution.
    """
    # Extract variables and bounds
    x, y = [var[0] for var in region]
    x_bounds, y_bounds = region

    # Define the potential function V(x, y) (governed by Laplace's equation)
    V = function('V')(x, y)

    # Define Laplace's equation: ∇²V = -current_input / resistance
    laplace_eq = diff(V, x, 2) + diff(V, y, 2) == -current_input / resistance

    # Solve the PDE with the given boundary conditions
    charge_distribution = desolve_laplace(laplace_eq, V, dvar=[x, y], ics=boundary_conditions)

    # Calculate the total charge (integral of charge density over the region)
    charge_density = -resistance * (diff(charge_distribution, x, 2) + diff(charge_distribution, y, 2))
    total_charge = integral(integral(charge_density, x_bounds), y_bounds)

    # Calculate the center of charge distribution using the centerOfMass function
    center_of_charge = centerOfMass(region, charge_density)

    return {
        'charge_distribution': charge_distribution,
        'total_charge': total_charge,
        'center_of_charge': center_of_charge
    }

In [51]:
# Define the region, resistance, current input, and boundary conditions
V = function('V')(x, y)
region = [(x, 0, 1), (y, 0, 2)]
resistance = 1  # Uniform resistance
current_input = 10  # Constant current input
boundary_conditions = {V(0, y): 0, V(1, y): 0, V(x, 0): 0, V(x, 2): 10}  # Example BCs

# Simulate the charge distribution
result = simulateChargeDistribution(region, resistance, current_input, boundary_conditions)

# Output results
print("Charge Distribution:", result['charge_distribution'])
print("Total Charge:", result['total_charge'])
print("Center of Charge:", result['center_of_charge'])

TypeError: desolve_laplace() got multiple values for argument 'dvar'